<a href="https://colab.research.google.com/github/Matinsalami/DataScience/blob/main/Dimensionality_Reduction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Main Approaches for Dimensionality Reduction


# Projection

Although the instances have too many feaatures, most of the features are correlated and some others are constants. So, we can figure out that the all the instances lie in a lower-dimensional space which is a subset of the higher-dimensional space.

Example: Most of the instances in a 3-D space are close to a 2-D plane which is a lower-Dimensionality space.
                                                                                             

## Manifold Learning

Projection is not the optimal solution in some cases. Sometimes the subspace may twist and turn. We have this in Swiss roll toy datset.

Projection in this case may squash different layers. So we should unroll the Swiss roll to obtain the 2D dataset.

## PCA

PCA by far is the most popular dimensionality reduction algorithm. First it identifies the hyperplane closest to data, then it projects the data onto it.

## Preserving the variance

Before projecting onto a hyperplane, we must be able to find the hyperplane itself!

We should be careful to choose a hyperplane that preserves the **maximum** amount of variance in the data set, otherwise we may loose a lot of information. Another way to justify our choice of hyperplane is to find a hyperplane to **minimize** the mean square error of the data.

But what is **prinipal component**?

